**Introduction**

The aim of this exercise is to code a market-making algorithmic strategy to auto-response request for quotes (RfQs) in bonds in a multi-dealer to client platform. The model is based on the foundational Avellaneda and Stoikov model, with some adaptations to be used in the RfQ context. The core of the algorithm is a model that estimates the probability of winning the RfQ given the price we are quoting and the context of the operation.

Delivery: Jupyter notebook (Python) with all the compulsory parts of the exercise and results


***Description of the dataset***

rfqs.csv:
* date_time: date and time at which the quote is requested.
* Instrument: the bond for which the customer has requested a price
* client: client code (anonymized)
* price: price quoted to the client by the bank
* mid: market mid-price of the bond captured by the bank's system at the time of the operation
* vol_MM: amount requested by the client (in millions of euros).
* dv01: sensitivity of the bond to variations in its yield (a measure of risk of the bond)
* num_dealers: number of banks from whom the client has requested a quote
* side: 1 if it is buy -1 if it is sell (from the point of view of the bank, not the client)
* won: 0 if the bank did not close the operation, 1 if it did.

**Imports**

In [ ]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
# YOUR IMPORTS HERE

**Data reading and preparation**

Read both datasets and perform an exploratory analysis of the data. Look at potential issues of missing data or outliers and clean it if you consider it necessary

In [ ]:
rfq_df = pd.read_csv("rfqs.csv", index_col = None)[['DateTime', 'instrument', 'client', 'price', 'mid',
       'vol(MM)', 'dv01', 'num_dealers', 'side', 'won']]

Clients are not really sensitive to the mid-price when they are buying or selling a product, since when they request a quote they typically have an estimation of it and therefore they are only sensitive to the "spread" we charge them on top of this mid-price as a compensation for the service of liquidity provision. The only exception is when they are requesting quotes for "price discovery", i.e. they are not interested to trade with any dealer, only to know which prices are quoted, but we will not consider this case in the exercise. To calculate the spread feature:

1) Calculate the spread as the difference between the price and the mid, taking into account the side so that the spread should be mostly positive (there might be cases in which is negative, but a minority, for very aggressive quoting).

2) Plot the histogram of the spreads and remove potential extreme cases (by business knowledge, we don't expect spreads higher than 10% of the mid price in absolute value)

Create the following time features:
* date column, using the DateTime column
* time column, using the DateTime column
* periodOfDay column, using the DateTime column to create a categorical feature that specifies the time of the day (morning, afternoon, evening, night)

**Probability of winning the RfQ**

Let us implement a model that predicts the probability of winning the RfQ given our quoted spread. The target feature is the binary variable "won", which has values 1 if we win the RfQ and 0 if not. This is therefore a classification problem. In order to use this model in the Avellaneda - Stoikov market-making algorithm, we need to implement a predictive model with the following characteristics:

1) the output is a probability, not the label

2) the probability has to be a monotonic decreasing function of the quoted spread, since otherwise the result would not have and economic sense: if we quote a worse price to the client, the probability of winning the RfQ has to decrease necessarily!

3) exploit other features of the dataset like client, instrument, volume, dv01, side or number of dealers or any derived feature from those.

A possible model that fulfills these characteristics is the Logistic Regression, but you can choose any other model that respects 1-3. In the following, separate the dataset in training and test. If you are going to finetune hyperparameters, you can add a validation set or perform cross-validation over the training set. We recommend to analyze the following points when building the model:

* For this exercise, limit the features to train the model to the following: 'instrument', 'client', 'vol(MM)', 'dv01','num_dealers', 'side', 'spread','periodOfDay' (and your target "won")

* Pre-process your features before building your model: many models (not all, for instance trees) require that categorical features are one-hot encoded. The performance of many models also improve when normalizing the continuous features.

* Use feature selection algorithms to reduce the input space before applying your model, unless you use a model that has feature selection embedded (e.g. in the Logistic Regression you can use an L1 Lasso regularization). Discuss which features are more relevant.

* Even if your model enforces the monotonicity with respect to the spread, test graphically that this is the case

* Be careful about imbalance in the dataset, how can you handle imbalance using your model?

* Check that indeed the probability is a decreasing function of the spread

* As quality metrics to test the model, use the following, and interpret the result:
    * Accuracy. If the dataset is imbalanced, this is a potentially missleading metric. To see it, compare the results of your model with a baseline model where the majority class is always predicted. You can use sklearn's DummyClassifier for this
    * Confusion matrix
    * Area under the ROC curve
    * Precision - Recall curve
    * Reliability curve (also called calibration curve): divide the model output probability space in buckets of 10%, e.g. 0-10%, 10-20%, ..., 90-100%. Now for each bucket of output probabilities, compute the percentage of actual won operations (so-called "hit-miss"). If the model was perfect, you would obtain a straight line. Plot the curve, and also calculate the so-called Brier score, which is an average of squares of distances between the mid-points of the buckets and the hit-misses weighted by the number of observations per bucket. You can implement it yourself or use calibration_curve from sklearn.calibration
    
* Check how the performance depends on the amount of days used to train the model and select a good value to be used in the second part of the exercise

[Hint] When doing the analysis, in principle, you should preserve the temporal ordering being a time-series data. However, once you condition on the features, the observations become relatively independent (you can check the autocorrelation for instance) and you can in principle take random subsets, which can help since the data is quite imbalanced and sometimes you might get subsets with only the majority class.

[Tip] Once you have a pipeline of feature transformation, selection and model fit, use python's pipeline class to wrap it up together. You can use the following libraries:

* sklearn.compose.ColumnTransformer: to transform separately your categorical and numerical features, for instance one-hot encoding the categorical and standarizing the numerical:

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])
    
* sklearn.preprocessing: it contains multiple pre-processors like OneHotEncoder to use in ColumnTransformer
* sklearn.feature_selection.SelectFromModel: you can use it for feature selection using a model like Lasso for example
* sklearn.pipeline.Pipeline: finally you put all together in a pipeline, for instance:

pipeline = Pipeline([('preprocessor', preprocessor),
                 ('feature_selector', selector),
                 ('model', model)])

**Avellaneda-Stoikov market-making algorithm for RfQs**

The Avellaneda-Stoikov model was originally developed for market-making in a Limit Order Book (LOB), although in practice is more suited to RfQ protocols, since it does not really model the LOB: for instance, it considers that any price is possible, ignoring the tick size. The optimal quoting (limit order price in a LOB, in our case the price quoted for the RfQ) has two components: one that considers the inventory risk, i.e. the risk of depreciation of our inventory until we find another investor willing to trade in the opposite direction; and the other that considers the price-sensitivity of the investor, via the probability of winning the RfQ model implemented above (in the case of a LOB, this is a filling probability or our limit order). For simplicity, we will use the first-order approximation from the original article, which simplifies the inventory risk term. The pricing formulae for the bid and ask half-spreads read:

$\delta_{b,i} = \gamma \sigma^2 (T-t)(q+\frac{v_i}{2})+\frac{1}{\gamma v_i} \log(1-\gamma v_i\frac{P(\delta_{b,i})}{P'(\delta_{b,i})})$

$\delta_{a,i} = \gamma \sigma^2 (T-t)(-q+\frac{v_i}{2})+\frac{1}{\gamma v_i} \log(1-\gamma v_i\frac{P(\delta_{a,i})}{P'(\delta_{a,i})})$

Recall that the actual price quoted is the full-price:

$p_{b,i} = p_{mid,i} - \delta_{b,i}$

$p_{a,i} = p_{mid,i} + \delta_{a,i}$

In these formulae, we have:

* $\delta_{b,i}, \delta_{a,i}$: the half-spreads quoted for bid RfQs (the client wants to sell) and ask RfQs (the client wants to buy), respectively.
* $\gamma$: the risk-aversion of the trader or business operating the algorithm. It is an input to the algorithm.
* $\sigma$: the volatility (standard deviation of returns) of the instrument. It has to be computed at the same time-scale used in the model (e.g. if time is quoted in seconds, then it has to be a volatility per second)
* $T$: the time-horizon in which we expect the market-maker to close any open position (e.g. inventory = 0). In this exercise we will consider a market-making operating during trading hours and closing all positions at market-close, to avoid overnight risk.
* $t$: the current time at which we are quoting the RfQ. We will take as reference $t=0$ the opening of the market.
* $v_i$: the volume of the RfQ, where $i$ is the index of the RfQ. We will use the units of the dataset in MM€.
* $q$: the inventory held in the bond. We consider a separate inventory for each of the instruments. As a simplification, we don't consider correlations between instruments, which would require a multivariate version of the market-making formula. The inventory is $q=0$ at the beginning of the day. When a trade is closed, if the bank buys then $q \rightarrow q + v_i$, and if it sells $q \rightarrow q - v_i$. We allow negative inventories which imply a short position in the instrument.
* $P()$: the predictive probability of winning the RfQ using our model. Although the formula only shows as input the half-spreads, it can also be fed with any other feature available at quotiong time.
* $P'()$: the derivative of the probability of winning the RfQ with respect to the half-spread. For some models like the Logistic Regression it can be computed analytically, or you can use the utility function provided in the class.


Write a class for the Avellaneda-Stoikov market-making algorithm by filling the interface (class) below. The algorithm is initialized with the algo parameters, in our case:
* gamma, the risk aversion level
* the number of days used for training the models, that you have calculated in the previous section (it has a default value just in case)
* the time of closing the market as a datetime.time(). These quote-driven markets remain open longer than traditional stock exchanges, so we use 19:00:00 as a default value. Bear in mind that the algorithm works with a reference time normalized to 1, meaning that $t = 0$ at 00:00:00 and $t = 1$ at 23:59:59

You need to fill the following methods (don't change the inputs!!):
* _initPipeline: here you need you place your pipeline code from the previous section. The method has to be self-contained, i.e. all the info needed for initializing the pipeline must go into the method. Finally, store the pipeline in self.pipeline
* _getVolatility: here you calculate the end of day volatility for each of the instruments, storing in self.volatility a dictionary key = instrument names, value = volatility (std of end of day prices). For eod prices just take the last value observed in the day for each instrument.
* beforeMarketOpen: in this method you need to fit the pipeline, truncating the historical days to the number of days passed as an argument to the algorithm. This function is called every day before the market opens to prepare the algorithm for trading.
* _pricingFormula: here you need to implement the Avellaneda - Stoikov pricing formula, taking into account if we have a bid or ask request and using the inputs passed to the function. Your function should address problematic cases like denominators close or equal to zero.
* onTradeEvent: if the rfq is won, this function is triggered. You need to update the inventory and the cash (if we buy then inventory goes up by the volume of the rfq, and cash is reduced by volume x price, and the reverse if we sell)
* onMarketClose: at the end of the day all the inventory is liquidated at end of day prices passed as argument, meaning that you need to change the cash by the value of the inventory and set inventory for all instruments to zero

In [ ]:
# Complete the class only where it is specified by YOUR CODE HERE. Leave other parts untouched
# IMPORTANT: all inputs to the class are specified in the constructor, in the inputs of the methods, or internally
# Do not reference variables outside the scope of the class
from pandas.tseries.offsets import BDay
from scipy.optimize import fsolve

class AvellanedaStoikov():
    def __init__(self, gamma, num_days_training = 100, end_time = pd.to_datetime("19:00:00").time()):
        # Constructor of the class
        self.gamma = gamma
        self.num_days_training = num_days_training
        self.end_time = end_time

    def copy(self):
        return AvellanedaStoikov(self.gamma, self.num_days_training, self.end_time)

    def _initAlgo(self, instrument_names):
        # Helper method that initializes the state of the algorithm
        self.cash = 0
        self._initPipeline()
        self._initInventory(instrument_names)

    def _initInventory(self, instrument_names):
        # Helper method to initialize to zero the inventory for each instrument
        q_list = [0]*len(instrument_names)
        self.inventory = dict(zip(instrument_names, q_list))

    def _initPipeline(self):
        # In this helper method you need to RECREATE the pipeline you have selected during the first part
        # of the exercise. You have to store the pipeline in self.pipeline class variable at the end of the method
        # YOUR CODE HERE

    def _fractionOfDay(self, dt):
        # Helper method to calculate fractions of days
        return (dt.hour * 3600 + dt.minute * 60 + dt.second) / 86400

    def _getVolatility(self, historical_df):
        # Implement a function that calculates the volatility of differences in mid prices end of day
        # First you need to find the last mid per day, then calculate differences and
        # finally the standard deviation (volatility). The result is a dictionary instrument - volatility
        # that is returned by the function
        # YOUR CODE HERE
        pass

    def _getDerivativeModel(self, model, context, feature):
        # Helper method to calculate the derivative of a model with respect to a feature
        epsilon = 0.001
        local_context = context.copy()
        predict0 = model.predict_proba(local_context)[0][1]
        local_context.at[context.index[0], feature] += epsilon
        predict1 = model.predict_proba(local_context)[0][1]
        return (predict1 - predict0) / epsilon

    def beforeMarketOpen(self, training_df):
        # This method is called every day before the market opens
        self._initAlgo(training_df["instrument"].unique())
        # Truncate the training_df to the number of days you will use (specified in class variable
        # self.num_days_training), select the features and target to train your model, and call the fit
        # function of your model
        # YOUR CODE HERE
        # ****
        # END OF YOUR CODE
        # Additionally, we calculate the volatility for each instrument
        self.volatility = self._getVolatility(training_df)

    def _pricingFormula(self, side, time, inventory, volatility, volume, prob, prob_der):
        # evaluate the Avellaneda Stoikov formula given the inputs USING THE FORMUKA PROVIDED ABOVE
        # We recommend to handle cases where the formula might not work well, e.g. with derivatives close to zero
        # The function must return the Avellaneda - Stoikov SPREAD (not the price!)
        # In order to compute the time difference (t-T) use the following helper function:
        # self._fractionOfDay(time)
        # self._fractionOfDay(self.end_time)
        # YOUR CODE HERE
        pass

    def _iterPrice(self, rfq_model_df, spread):
        # We use this helper function to calculate the inputs of the pricing formula
        rfq_index = rfq_model_df.index[0]
        rfq_model_df.at[rfq_index, "spread"] = spread
        prob = self.pipeline.predict_proba(rfq_model_df)[0][1]
        prob_der = self._getDerivativeModel(self.pipeline, rfq_model_df, "spread")
        side = rfq_df.at[rfq_index, "side"]
        time = rfq_df.at[rfq_index, "time"]
        volume = rfq_df.at[rfq_index, "vol(MM)"]
        instrument = rfq_df.at[rfq_index, "instrument"]
        volatility = self.volatility[instrument]
        inventory = self.inventory[instrument]
        return self._pricingFormula(side, time, inventory, volatility, volume, prob, prob_der) - spread

    def onRfQevent(self, rfq_df):
        # In this code we take the rfq dataframe and calculate the optimal spread
        # We use exceptions to capture issues in the calculation, in this case as a safety we quote conservatily
        rfq_index = rfq_df.index[0]
        f = lambda x: self._iterPrice(rfq_df, x)
        try:
            spread = fsolve(f, 0)[0]
        except:
            print("Error in the calculation, quoting conservatively")
            spread = 1000 # if any issue quote conservatively
        return spread

    def onTradeEvent(self, rfq_df, price):
        # If there is a trade you must add or sustract (depending on side) the volume to the
        # inventory of the instrument that has been closed
        # To get propoerties from the dataframe, you can use the following code example:
        # rfq_index = rfq_df.index[0]
        # volume = rfq_df.at[rfq_index, "vol(MM)"]
        # YOUR CODE HERE, recall that class variables are called with self.variable_name
        # ****
        # You also need to change the cash in the opposite direction and taking into account the price
        # Prices of bonds are quoted in base 100, need to divide by 100 to convert to MM €
        # YOUR CODE HERE
        # ****

    def onMarketClose(self, prices_dict):
        # When the market closes, all the positions in the inventory are liquidated
        # meaning that you need to convert them to cash using prices and set inventory to zero
        # You should handle the possibility that no eod prices are available for the instrument
        # using prices_dict.get(key, 0.0) in the incoming dictionary to retrieve value having a
        # default 0.0
        # Prices of bonds are quoted in base 100, need to divide by 100 to convert to MM €
        # YOUR CODE HERE
        # ****

    def getCash(self):
        return self.cash

    def getInventory(self, instrument_name):
        return self.inventory[instrument_name]



**Backtesting environment**

The following code is a simple backtesting environment to run your market-making algorithm. Feel free to analyse the code but do not change the implementation

In [ ]:
import random
import warnings
warnings.filterwarnings('ignore')

class BacktestingSession():
    def __init__(self, algo, seed = 10):
        self.algo = algo
        random.seed(seed)

    def _rfq_won(self, model, rfq, spread):
        rfq_new_price = rfq.copy()
        rfq_index = rfq_new_price.index[0]
        rfq_new_price.at[rfq_index, "spread"] = spread
        prob = model.predict_proba(rfq_new_price)[0][1]
        return prob > random.random()

    def _get_eod_prices(self, rfq_df):
        return rfq_df[["instrument", "mid"]].groupby("instrument").last()["mid"].to_dict()

    def run_backtest(self, backtest_df, backtest_days):
        required_columns = set(["instrument", "client", "mid", "vol(MM)", "dv01", "num_dealers", "side", "date", "time", "periodOfDay"])
        if not required_columns.issubset(backtest_df.columns):
            return "Invalid backtest dataframe. Review that the required input columns are present"
        profit_and_loss = []
        columns_results = ["rfq_id", "price_algo", "spread_algo", "won_algo", "inventory_algo"]
        algo_results_df = pd.DataFrame(columns = columns_results)
        # loop over dates
        for date in backtest_days:
            # initialize algo before the market opens
            historical_df = backtest_df[backtest_df["date"] < date]
            self.algo.beforeMarketOpen(historical_df)
            # loop over all rfqs of the day
            single_day_df = backtest_df[backtest_df["date"] == date]
            rfq_id, prices_algo, spreads_algo, inv_algo, won_algo = [], [],[],[],[]
            for index, _ in single_day_df.iterrows():
                rfq_id.append(int(index))
                rfq = single_day_df[single_day_df.index == index]
                # get a spread quote from the algo
                spread = self.algo.onRfQevent(rfq)
                spreads_algo.append(spread)
                instrument_name = rfq_df.at[index, "instrument"]
                inv_algo.append(self.algo.getInventory(instrument_name))
                mid = rfq_df.at[index, "mid"]
                side = rfq_df.at[index, "side"]
                price = mid - side * spread
                prices_algo.append(price)
                # simulate the result of the RfQ using the model from the algo
                rfq_was_won = self._rfq_won(self.algo.pipeline, rfq, spread)
                won_algo.append(int(rfq_was_won))
                if rfq_was_won:
                    self.algo.onTradeEvent(rfq, price)
            # end of day we force the algo to liquidate the inventory at eod prices (no market impact)
            eod_prices = self._get_eod_prices(single_day_df)
            self.algo.onMarketClose(eod_prices)
            profit_and_loss.append(self.algo.getCash())
            results_day_df = pd.DataFrame({"rfq_id": rfq_id, "price_algo": prices_algo, "spread_algo": spreads_algo, \
                                        "won_algo": won_algo, "inventory_algo": inv_algo})
            algo_results_df = pd.concat([algo_results_df, results_day_df])
        # wrap up results
        pl_df = pd.DataFrame({"date": backtest_days, "P&L": profit_and_loss})
        test_set = backtest_df[backtest_df["date"].isin(backtest_dates)]
        rfq_results_df = test_set.join(algo_results_df.set_index("rfq_id"))
        return pl_df, rfq_results_df

    def run_pl_backtest_gamma(self, backtest_df, backtest_days, gamma_list):
        pl_results_df = None
        rfq_df_list = []
        for gamma in gamma_list:
            print(gamma)
            gamma_algo = self.algo.copy()
            gamma_algo.gamma = gamma
            gamma_backtest = BacktestingSession(gamma_algo)
            pl_df, rfq_df = gamma_backtest.run_backtest(backtest_df, backtest_days)
            pl_df = pl_df.rename(columns = {"P&L": str(gamma)})
            if pl_results_df is None:
                pl_results_df = pl_df
            else:
                pl_results_df = pd.merge(pl_results_df, pl_df, on = "date", how = "left")
            rfq_df_list.append(rfq_df)
        return pl_results_df, rfq_df_list

**Backtesting of the trading algorithm**

Now you will use a backtesting utility to analyze the performance of your trading algorithm. As a performance metric we will use the profit & loss (P&L) distribution at the end of the day. This is computed by keeping track of a cash account that is 0 at the beginning of the day (we consider we can borrow money at 0% interest rate). When we buy an instrument, the cash account decreases $-p_{a,i} * v_i$, whereas when we sell we add $p_{b_i} * v_i$. If the algorithm has any residual inventory at the end of the day, we assume it can be sold (if long) or bought (if short) at the closing mid-price (which is an optimistic hypothesis). Therefore, the cash account at then end of the day reflects the profit or loss of that day.

At the core of the backtesting engine is the simulation of the RfQ result. We cannot use the historical result, since in backtesting we will be likely quoting a different price than the one quoted in the historical dataset. In this exercise, we will use the same model you have built for the probability of winning the RfQ to decide the result of the process, that is encapsulated in the Avellaneda Stoikov class. Bear in mind that this is a quite optimistic assumption, since in reality our model will be only an approximation (in the best scenario) of the actual behavior of investors! To simulate the process, for every price quoted, the backtesting engine calculates the probability of winning the RfQ with the model and generate a random uniform variabe between 0 and 1. If the random variable is lower than the probability, the RfQ is won.

Perform the following analysis:
* Run the backtesting over the same test set you selected to assess the probability of winning the RfQ model. The backtesting session needs and instance of the algo to be initialized. To run a backtesting,  you need to pass all the rfqs (train plus test) as a dataframe with at least the columns: instrument, client, mid, vol(MM), dv01, num_dealers, side, date, time, periodOfDay + a numerical index for each rfq (this can just be the index assigned by Pandas to each row). Additionally, you need to pass a list of dates over which testing will be done (i.e. the dates of your test set)

* The backtesting returns two dataframes: one with the profit&loss for each day of the backtesting period, and an enriched test set with additional columns related to the outputs of the simulation, with columns:
    * spread_algo: the spread quoted by the trading algorithm for the given RfQ
    * price_algo: the price quoted by the trading algorithm for the given RfQ (mid + side * spread)
    * won_algo: if the trading algorithm won the RfQ in the simulation (as in the training set, 1 = won, 0 = miss)
    * inventory_algo: the inventory of the algo before quoting the RfQ, in the instrument being quoted (since we have multiple instruments, the algo keeps and inventory of each instrument separately)

* Plot the distribution of daily P&L for the algorithm and different values of the risk-aversion. Compute the mean, standard deviation, Sharpe ratio and the maximum drawdown using the backtesting results. For the latter you need to calculate the accumulated return of the strategy over the backtesting period: assume you start with initial holdings "1" and compound the daily return over this initial holding (e.g. if the first day the strategy generates 3%, then end of day the holdings are 1.03, if second day is -2%, then you have 1.0094, and so on).

* Calculate the distribution of the daily hit&miss (H&M), which is the ratio between rfqs won and total rfqs. Do you see any correlation between hit & miss and P&L.

* Rerun the analysis using different values of gamma in a logarithmic scale. You can use the utility provided in the backtesting framework. Compare the statistics (P&L, H&M) from the previous point for the different values of gamma. How do you think the risk aversion affects the P&L distribution? And the H&M?
    
    [Optional] Analyze the limit cases of zero and infinite risk aversion in the Avellaneda - Stoikov formula and interpret the results
    
    [Optional] If you enjoy mathematics, you can derive the target H&M of the model using a simple model for the probability of winning the RfQ. Assume $P(win|\delta) \equiv P(\delta) = exp(-\alpha \delta)$, and plug it in the Avellaneda - Stoikov formulae. You will see that the formulae simplify and you have a closed-form for the spread. The target hit & miss corresponds to evaluationg $P(\delta)$ with the optimal spread you just derived. Analyze the result and dependencies with respect to the parameters.

* Picking one particular gamma, analyze days where the algorithm performs the best, the worse, and average. Looking at the behaviour of the algo during the day for the different instruments, try to explain the reason of the good / bad / average performance. Support your discussions with plots if needed (evolution of inventory levels, spreads, mids, etc)

In [ ]:
## Example running code
## rfq_df is the full dataset of rfqs, backtest_dates is a list of test dates (a subset of the dates in rfq_df)
# algo = AvellanedaStoikov(gamma = 0.01)
# backtesting = BacktestingSession(algo)
# pl_result_df, rfq_result_df = backtesting.run_backtest(rfq_df, backtest_dates)